In [1]:
import pandas as pd
import causaldata
from scipy.stats import chi2_contingency
from itertools import combinations
from pgmpy.estimators.CITests import pillai_trace
from pyitlib import discrete_random_variable as drv


# The dataset itself

In [2]:
df = causaldata.ccdrug.load_pandas().data
df

,custody,male,first_offense,age,offense,prev_convictions,drg_class,drg_culpability,drg_increasing_ser_stat_2,drg_increasing_ser_stat_3,...,drg_reducing_ser_7,drg_reducing_ser_8,drg_reducing_ser_9,drg_reducing_ser_10,drg_reducing_ser_11,drg_reducing_ser_12,drg_reducing_ser_13,drg_reducing_ser_14,drg_reducing_ser_15,drg_reducing_ser_16
0,1,1,1,25 to 34,Supplying,4 to 9,Cannabis or Cannabis resin,Significant role,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,1,1,25 to 34,Possession with intent to supply,None,Cannabis or Cannabis resin,Significant role,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,1,0,25 to 34,Possession with intent to supply,4 to 9,Cannabis or Cannabis resin,Significant role,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,1,35 to 44,Possession with intent to supply,None,Cannabis or Cannabis resin,Significant role,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,1,0,25 to 34,Possession with intent to supply,None,Cannabis or Cannabis resin,Significant role,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16968,0,1,1,25 to 34,Production/being concerned in production/culti...,None,Cannabis or Cannabis resin,Lesser role,0,0,...,0,0,0,0,0,0,0,0,0,0
16969,1,1,1,25 to 34,Production/being concerned in production/culti...,None,Cannabis or Cannabis resin,Lesser role,0,0,...,0,0,1,0,0,0,0,0,0,0
16970,1,1,0,45 to 54,Possession with intent to supply,10 or more,Cocaine,Lesser role,0,0,...,1,0,0,0,0,0,0,0,0,0
16971,1,1,0,35 to 44,Possession with intent to supply,10 or more,Cocaine,Lesser role,0,0,...,1,0,0,0,0,0,0,0,0,0


In [3]:
for column in df.columns:
    print(column, df[column].value_counts())
    print("")

custody custody
1    9355
0    7618
Name: count, dtype: int64

male male
1    15737
0     1236
Name: count, dtype: int64

first_offense first_offense
1    10596
0     6377
Name: count, dtype: int64

age age
25 to 34    6149
18 to 24    5407
35 to 44    3208
45 to 54    1731
Over 54      478
Name: count, dtype: int64

offense offense
Possession with intent to supply                        8389
Production/being concerned in production/cultivation    4713
Supplying                                               3276
Bringing in/taking out controlled drug                   595
Name: count, dtype: int64

prev_convictions prev_convictions
None          10853
1 to 3         4499
4 to 9         1143
10 or more      478
Name: count, dtype: int64

drg_class drg_class
Cannabis or Cannabis resin    8459
Cocaine                       4185
Heroin                        2539
Other Class B                 1020
Other Class A                  507
Class C                        263
Name: count, dtype: int

# Independence Test through CHI^2 test
Low P value means reject independence, it is also symmetric

In [4]:
def generate_contigency_table(df, x, y):
    return pd.crosstab(df[x], df[y]).values

all_pairs = [(column, "custody")for column in df.columns if column != "custody" ]
# all_pairs = list(combinations(df.columns, 2))

In [5]:
results = {k: dict() for k in df.columns}

for x, y in all_pairs:
    table = generate_contigency_table(df, x, y)
    chi2, p, dof, expected = chi2_contingency(table)
    if p > 0.05:
        print(x, y, p)
    results[x][y] = p

drg_increasing_ser_other_18 custody 0.15638480811597355
drg_reducing_ser_3 custody 0.964609225711807


In [6]:
results_df = pd.DataFrame(results)
results_df

,custody,male,first_offense,age,offense,prev_convictions,drg_class,drg_culpability,drg_increasing_ser_stat_2,drg_increasing_ser_stat_3,...,drg_reducing_ser_7,drg_reducing_ser_8,drg_reducing_ser_9,drg_reducing_ser_10,drg_reducing_ser_11,drg_reducing_ser_12,drg_reducing_ser_13,drg_reducing_ser_14,drg_reducing_ser_15,drg_reducing_ser_16
custody,NaN,6.768523e-53,3.114775e-25,0.000082,2.549434e-143,8.381681e-156,0.0,1.765227e-122,0.000033,1.732018e-34,...,0.001509,1.353348e-159,6.153782e-79,2.962734e-190,7.929513e-42,9.675272e-16,7.014575e-23,6.694878e-50,6.509789e-53,8.341767e-59


# Independence test pillai_trace from notebook (?)
To avoid confusion in the output of `pillai_trace`, note that output is `True` if the p-value _exceeds_ the significance level, which is the opposite of what you might be used to. This is because `True` refers to _independence being accepted_ instead of _the null hypothesis being rejected_. In the context of causal discovery covered later in the course, this is an approach to hypothesis testing that you will see more often. 

In [7]:
categorical_variables = ["age", "offense", "prev_convictions", "drg_class", "drg_culpability"]
for column in categorical_variables:
    df[column] = df[column].astype("category").cat.codes

In [8]:
for x, y in all_pairs:
    value = pillai_trace(x, y, Z=[], data=df, significance_level=0.05)
    if value:
        print(x, y, value)

age custody True
drg_increasing_ser_other_18 custody True
drg_reducing_ser_3 custody True


# Independence test through (conditional) mutual information
No P value

In [9]:
for x, y in all_pairs:
    I_xy = drv.information_mutual(df[x].to_numpy(), df[y].to_numpy(), base=2) # add Z for conditional
    if I_xy > 0.05:
        print(x, y, I_xy)

drg_class custody 0.13834876031401544
